# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods that will be used to get training and validation data
loader.

You will have to write a custom dataset class to load data.

**Note that there are not separate validation data, so you will have to create your validation set by dividing train data into train and validation data. Usually, in practice, we do `80:20` ratio for train and validation, respectively.** 

For example,

```
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    
    
```

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

In [ ]:
import os
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
#from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import lr_scheduler

from trainer import Trainer, hooks, configuration
from trainer.utils import setup_system, patch_configs
from trainer.metrics import AccuracyEstimator
from trainer.tensorboard_visualizer import TensorBoardVisualizer
from sklearn.utils.class_weight import compute_class_weight
from PIL import Image

In [ ]:
class KenyanFood13Dataset(torch.utils.data.Dataset):
    """
    This custom dataset class takes root directory, flag, and transform 
    and returns training dataset if flag is 0, validation dataset if flag is 1 
    else it returns test dataset.
    """
    def __init__(self, data_root, flag, split, transform, random_state):
        labels_to_idx = {'githeri' : 0, 'ugali' : 1, 'kachumbari' : 2, 'matoke' : 3, 
                         'sukumawiki' : 4, 'bhaji' : 5, 'mandazi' : 6, 
                         'kukuchoma' : 7, 'nyamachoma' : 8, 'pilau' : 9, 
                         'chapati' : 10, 'masalachips' : 11, 'mukimo' : 12}
        self.idx_to_labels = {0 : 'githeri', 1 : 'ugali', 2 : 'kachumbari', 3 : 'matoke',
                              4 : 'sukumawiki', 5 : 'bhaji', 6 : 'mandazi',
                              7 : 'kukuchoma', 8 : 'nyamachoma', 9 : 'pilau',
                              10 : 'chapati', 11 : 'masalachips', 12 : 'mukimo'}
        self.data_root = data_root
        self.transform = transform
        self.imgs = []
        self.labels = []
        self.weights = None
        
        if flag == 0 or flag == 1:
            
            data_csv = pd.read_csv(os.path.join(data_root, 'train.csv'))
            
            food_category = {}
            
            for idx, row in data_csv.iterrows():
                key = row['class']
                if key not in food_category.keys():
                    food_category[key] = []
                food_category[key].append(row['id'])
            
            num_classes = len(food_category.keys())
            assert num_classes == 13
            
            for category in food_category.keys():
                random.seed(random_state)
                random.shuffle(food_category[category])
                
                count = len(food_category[category])
                split_idx = math.floor(count*split)
                if flag == 0:
                    self.imgs.extend(food_category[category][:split_idx])
                    count = len(food_category[category][:split_idx])
                    self.labels.extend([labels_to_idx[category]]*count)
                else:
                    self.imgs.extend(food_category[category][split_idx:])
                    count = len(food_category[category][split_idx:])
                    self.labels.extend([labels_to_idx[category]]*count)
                
                self.weights = compute_class_weight('balanced', 
                                                    classes=np.unique(self.labels), 
                                                    y=self.labels)
        else: # flag == 2
            data_csv = pd.read_csv(os.path.join(data_root, 'test.csv'))
            self.imgs.extend(data_csv['id'])
            self.labels = None
    
    def __len__(self):
        """
        return length of the dataset
        """
        return len(self.imgs)

    def __getitem__(self, idx):
        """
        For given index, return images with resize and preprocessing.
        """
        img_name = str(self.imgs[idx]) + '.jpg'
        img_path = os.path.join(self.data_root, 'images', 'images', img_name)
        image = Image.open(img_path).convert("RGB")
        target = None
        if self.transform is not None:
            image = self.transform(image)
        if self.labels is not None:
            target = self.labels[idx]
        else:
            target = str(self.imgs[idx])
        return image, target

    def get_class_weight(self):
        return self.weights

    def idx_to_labels(self, idx):
        return self.idx_to_labels[idx]

## <font style="color:green">2. Configuration [5 Points]</font>

Define your configuration in this section.

For example,

```
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/pytorch-opencv-course-classification/" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [ ]:
from typing import Callable, Iterable
from dataclasses import dataclass

from torchvision import transforms

@dataclass
class SystemConfig:
    seed: int = 42  # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = False  # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True  # make cudnn deterministic (reproducible training)


@dataclass
class DatasetConfig:
    root_dir: str = "."  # dataset directory root
    split: float = 0.8
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    resiz = 256
    inpsz = 224

    train_transforms: Iterable[Callable] = transforms.Compose([
        #transforms.Resize(resiz),
        #transforms.RandomAffine(degrees=15, translate=(0.25,0.25), scale=(0.5,1.5)),
        #transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.RandomResizedCrop(inpsz),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(0.3,0.3,0.3),
        transforms.RandomAffine(degrees=15, translate=(0.25,0.25), scale=(0.5,1.5)),
        #transforms.CenterCrop(inpsz),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])  # data transformation to use during training data preparation
    test_transforms: Iterable[Callable] = transforms.Compose([
        transforms.Resize(resiz),
        transforms.CenterCrop(inpsz),
        transforms.ToTensor(), # this re-scales image tensor values between 0-1. image_tensor /= 255
        # subtract mean (0.485, 0.456, 0.406) and divide by variance (0.229, 0.224, 0.225)
        transforms.Normalize(mean, std),
    ])  # data transformation to use during test data preparation

@dataclass
class DataloaderConfig:
    batch_size: int = 32  # amount of data to pass through the network at each forward-backward iteration
    num_workers: int = 8  # number of concurrent processes using to prepare data


@dataclass
class OptimizerConfig:
    learning_rate: float = 0.0001  # determines the speed of network's weights update
    momentum: float = 0.9  # used to improve vanilla SGD algorithm and provide better handling of local minimas
    weight_decay: float = 0.0001  # amount of additional regularization on the weights values
    lr_step_milestones: Iterable = (
        30, 40
    )  # at which epoches should we make a "step" in learning rate (i.e. decrease it in some manner)
    lr_gamma: float = 0.1  # multiplier applied to current learning rate at each of lr_step_milestones


@dataclass
class TrainerConfig:
    model_dir: str = "checkpoints"  # directory to save model states
    model_saving_frequency: int = 1  # frequency of model state savings per epochs
    device: str = "cpu"  # device to use for training.
    epoch_num: int = 50  # number of times the whole dataset will be passed through the network
    progress_bar: bool = True  # enable progress bar visualization during train process

## <font style="color:green">3. Evaluation Metric [10 Points]</font>

Define methods or classes that will be used in model evaluation, for example, accuracy, f1-score, etc.

In [ ]:
# Same as the one used in Trainer (AccuracyEstimator, BaseMetric) : I did not change

In [ ]:
class AccuracyEstimator(BaseMetric):
    def __init__(self, topk=(1, )):
        self.topk = topk
        self.metrics = [AverageMeter() for i in range(len(topk) + 1)]

    def reset(self):
        for i in range(len(self.metrics)):
            self.metrics[i].reset()

    def update_value(self, pred, target):
        """Computes the precision@k for the specified values of k"""
        with torch.no_grad():
            maxk = max(self.topk)
            batch_size = target.size(0)

            _, pred = pred.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))

            for i, k in enumerate(self.topk):
                correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
                self.metrics[i].update(correct_k.mul_(100.0 / batch_size).item())

    def get_metric_value(self):
        metrics = {}
        for i, k in enumerate(self.topk):
            metrics["top{}".format(k)] = self.metrics[i].avg
        return metrics

## <font style="color:green">4. Train and Validation [5 Points]</font>

Write the methods or classes that will be used for training and validation.

In [ ]:
# Same as the one used in Trainer (hooks.py, trainer.py)
# I only changed fit to save the epoch # and loss to the model file name as shown below

In [ ]:
# **This is a generic class for training loop.**
#
# Trainer class is equivalent to the `main` method. 
#
# In the main method, we were passing configurations, the model, optimizer, 
# learning rate scheduler, and the number of epochs.  It was calling the method to get the 
# train and test data loader. Using these, it is training and validating the model. 
# During training and validation, it was also sending logs to TensorBoard and saving the model.
#
# The trainer class is doing the same in a more modular way so that we can experiment with 
# different loss functions, different visualizers, different types of targets, etc. 
#

"""Unified class to make training pipeline for deep neural networks."""
import os
import datetime

from typing import Union, Callable
from pathlib import Path
from operator import itemgetter

import torch
from tqdm import tqdm
#from tqdm.auto import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

from .hooks import test_hook_default, train_hook_default
from .visualizer import Visualizer


#
# Setting different attributes.
#
# **Parameters:**
# - `model` : `nn.Module` - torch model to train
# - `loader_train` : `torch.utils.DataLoader` - train dataset loader.
# - `loader_test` : `torch.utils.DataLoader` - test dataset loader
# - `loss_fn` : `callable` - loss function. In the main function, the cross-entropy loss was 
# being used; here, we can pass the loss we want to use. 
# For example, if we are solving a regression problem, we can not use cross-entropy loss. 
# It is better to use RMS-loss.
#
# - `metric_fn` : `callable` - evaluation metric function. In the main function, we had loss 
# and accuracy as our evaluation metric. Here we can pass any evaluation metric. 
# For example, in a detection problem, we need a precision-recall metric instead of accuracy.
#
# - `optimizer` : `torch.optim.Optimizer` - Optimizer.
# - `lr_scheduler` : `torch.optim.LrScheduler` - Learning Rate scheduler.
# - `configuration` : `TrainerConfiguration` - a set of training process parameters.
#
# Here, we need a data iterator and target iterator separately, because we are writing a 
# general trainer class. For example, for the detection problem for a single image, 
# we might have `n`-number of objects and their coordinates. 
#        
# - `data_getter` : `Callable` - function object to extract input data from the 
# sample prepared by dataloader.
#        
# - `target_getter` : `Callable` - function object to extract target data from the 
# sample prepared by dataloader.
#        
# - `visualizer` : `Visualizer` - optional, shows metrics values (various backends are possible). 
# We can pass the visualizer of our choice. 
# For example, Matplotlib based visualizer, TensorBoard based, etc.
#
# It is also calling its method `_register_default_hooks` what this method does we will see next. 
# In short, this is making sure that training and validation function is registered at the 
# time of trainer class object initiation. 
#
# It is calling the another method `register_hook` to register training (`train_hook_default`) 
# and validation (`test_hook_default`) functions. 
# `train_hook_default` and `test_hook_default` are defined in the `hook`-module.
#
# It is updating the key-value pair of a dictionary, where the key is string and value is a 
# callable function.
#
# **Parameters:**
#
# - `hook_type`: `string` - hook type. For example, wether the function will be used for train or test.
# - `hook_fn`: `callable` - hook function.
#
# Taking the number of epochs and training and validating the model. 
# It is also adding logs to the visualizer. 
#
# **Parameters:**
#
# - `epochs`: `int` - number of epochs to train model.
#

class Trainer:  # pylint: disable=too-many-instance-attributes
    """ Generic class for training loop.

    Parameters
    ----------
    model : nn.Module
        torch model to train
    loader_train : torch.utils.DataLoader
        train dataset loader.
    loader_test : torch.utils.DataLoader
        test dataset loader
    loss_fn : callable
        loss function
    metric_fn : callable
        evaluation metric function
    optimizer : torch.optim.Optimizer
        Optimizer
    lr_scheduler : torch.optim.LrScheduler
        Learning Rate scheduler
    configuration : TrainerConfiguration
        a set of training process parameters
    data_getter : Callable
        function object to extract input data from the sample prepared by dataloader.
    target_getter : Callable
        function object to extract target data from the sample prepared by dataloader.
    visualizer : Visualizer, optional
        shows metrics values (various backends are possible)
    # """
    def __init__( # pylint: disable=too-many-arguments
        self,
        model: torch.nn.Module,
        loader_train: torch.utils.data.DataLoader,
        loader_test: torch.utils.data.DataLoader,
        loss_fn: Callable,
        metric_fn: Callable,
        optimizer: torch.optim.Optimizer,
        lr_scheduler: Callable,
        device: Union[torch.device, str] = "cuda",
        model_saving_frequency: int = 1,
        save_dir: Union[str, Path] = "checkpoints",
        model_name_prefix: str = "model_",
        data_getter: Callable = itemgetter("image"),
        target_getter: Callable = itemgetter("target"),
        stage_progress: bool = True,
        visualizer: Union[Visualizer, None] = None,
        get_key_metric: Callable = itemgetter("top1"),
    ):
        self.model = model
        self.loader_train = loader_train
        self.loader_test = loader_test
        self.loss_fn = loss_fn
        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.lr_scheduler = lr_scheduler
        self.device = device
        self.model_saving_frequency = model_saving_frequency
        self.save_dir = save_dir
        self.model_name_prefix = model_name_prefix
        self.stage_progress = stage_progress
        self.data_getter = data_getter
        self.target_getter = target_getter
        self.hooks = {}
        self.visualizer = visualizer
        self.get_key_metric = get_key_metric
        self.metrics = {"epoch": [], "train_loss": [], "test_loss": [], "test_metric": []}
        self._register_default_hooks()

    def fit(self, epochs):
        """ Fit model method.

        Arguments:
            epochs (int): number of epochs to train model.
        """
        iterator = tqdm(range(epochs), dynamic_ncols=True)
        for epoch in iterator:
            output_train = self.hooks["train"](
                self.model,
                self.loader_train,
                self.loss_fn,
                self.optimizer,
                self.device,
                prefix="[{}/{}]".format(epoch, epochs),
                stage_progress=self.stage_progress,
                data_getter=self.data_getter,
                target_getter=self.target_getter
            )
            output_test = self.hooks["test"](
                self.model,
                self.loader_test,
                self.loss_fn,
                self.metric_fn,
                self.device,
                prefix="[{}/{}]".format(epoch, epochs),
                stage_progress=self.stage_progress,
                data_getter=self.data_getter,
                target_getter=self.target_getter,
                get_key_metric=self.get_key_metric
            )
            if self.visualizer:
                self.visualizer.update_charts(
                    None, output_train['loss'], output_test['metric'], output_test['loss'],
                    self.optimizer.param_groups[0]['lr'], epoch
                )

            self.metrics['epoch'].append(epoch)
            self.metrics['train_loss'].append(output_train['loss'])
            self.metrics['test_loss'].append(output_test['loss'])
            self.metrics['test_metric'].append(output_test['metric'])

            if self.lr_scheduler is not None:
                if isinstance(self.lr_scheduler, ReduceLROnPlateau):
                    self.lr_scheduler.step(output_train['loss'])
                else:
                    self.lr_scheduler.step()

            if self.hooks["end_epoch"] is not None:
                self.hooks["end_epoch"](iterator, epoch, output_train, output_test)

            if (epoch + 1) % self.model_saving_frequency == 0:
                os.makedirs(self.save_dir, exist_ok=True)
                fname = str(epoch) + '_{:.3f}'.format(output_test['loss'])
                torch.save(
                    self.model.state_dict(),
                    os.path.join(self.save_dir, self.model_name_prefix + fname) 
                )
        return self.metrics

    def fit0(self, epochs):
        """ Fit model method.

        Arguments:
            epochs (int): number of epochs to train model.
        """
        iterator = tqdm(range(epochs), dynamic_ncols=True)
        for epoch in iterator:
            output_train = self.hooks["train"](
                self.model,
                self.loader_train,
                self.loss_fn,
                self.optimizer,
                self.device,
                prefix="[{}/{}]".format(epoch, epochs),
                stage_progress=self.stage_progress,
                data_getter=self.data_getter,
                target_getter=self.target_getter
            )

            self.metrics['epoch'].append(epoch)
            self.metrics['train_loss'].append(output_train['loss'])

            if self.lr_scheduler is not None:
                if isinstance(self.lr_scheduler, ReduceLROnPlateau):
                    self.lr_scheduler.step(output_train['loss'])
                else:
                    self.lr_scheduler.step()

            if (epoch + 1) % self.model_saving_frequency == 0:
                os.makedirs(self.save_dir, exist_ok=True)
                fname = str(epoch)
                torch.save(
                    self.model.state_dict(),
                    os.path.join(self.save_dir, self.model_name_prefix + fname) 
                )
        return self.metrics

    def register_hook(self, hook_type, hook_fn):
        """ Register hook method.

        Arguments:
            hook_type (string): hook type.
            hook_fn (callable): hook function.
        """
        self.hooks[hook_type] = hook_fn

    def _register_default_hooks(self):
        self.register_hook("train", train_hook_default)
        self.register_hook("test", test_hook_default)
        self.register_hook("end_epoch", None)


## <font style="color:green">5. Model [5 Points]</font>

Define your model in this section.

**You are allowed to use any pre-trained model.**

In [ ]:
#
# Get the pretrained model:
def pretrained_resnext50(pretrained=True, fine_tune_start=1, num_class=13):
    resnet = models.resnext50_32x4d(pretrained=pretrained)
    
    if pretrained:
        for param in resnet.parameters():
            param.requires_grad = False

    if pretrained:
        if fine_tune_start <= 1:
            for param in resnet.layer1.parameters():
                param.requires_grad = True
        if fine_tune_start <= 2:
            for param in resnet.layer2.parameters():
                param.requires_grad = True
        if fine_tune_start <= 3:
            for param in resnet.layer3.parameters():
                param.requires_grad = True
        if fine_tune_start <= 4:
            for param in resnet.layer4.parameters():
                param.requires_grad = True
                
    last_layer_in = resnet.fc.in_features
    resnet.fc = nn.Sequential(nn.Linear(last_layer_in, 256),
                              nn.ReLU(inplace=True),
                              nn.Dropout(0.5),
                              nn.Linear(256, 32),
                              nn.ReLU(inplace=True),
                              nn.Dropout(0.2),
                              nn.Linear(32, num_class))
    
    return resnet

## <font style="color:green">6. Utils [5 Points]</font>

Define your methods or classes which are not covered in the above sections.

In [ ]:
# same as those in trainer - visualizer, utils, tensorboard_visualizer

## <font style="color:green">7. Experiment [5 Points]</font>

Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.

In [ ]:
class Experiment:
    def __init__(
        self,
        system_config: configuration.SystemConfig = configuration.SystemConfig(),
        dataset_config: configuration.DatasetConfig = configuration.DatasetConfig(),
        dataloader_config: configuration.DataloaderConfig = configuration.DataloaderConfig(),
        optimizer_config: configuration.OptimizerConfig = configuration.OptimizerConfig()
    ):

        # train dataloader
        train_dataset = KenyanFood13Dataset(dataset_config.root_dir, flag=0, 
                                            split=dataset_config.split,
                                            transform=dataset_config.train_transforms, 
                                            random_state=system_config.seed)
        class_weight = train_dataset.get_class_weight()
        self.loader_train = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=dataloader_config.batch_size,
            shuffle=True,
            num_workers=dataloader_config.num_workers
        )
        
        # validation dataloader
        val_dataset = KenyanFood13Dataset(dataset_config.root_dir, flag=1, 
                                          split=dataset_config.split,
                                          transform=dataset_config.test_transforms, 
                                          random_state=system_config.seed)
        self.loader_test = torch.utils.data.DataLoader(
            val_dataset,
            batch_size=dataloader_config.batch_size,
            shuffle=False,
            num_workers=dataloader_config.num_workers
        )


        setup_system(system_config)
        self.model = pretrained_resnext50(pretrained=True, fine_tune_start=4)

        self.loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight))
        self.metric_fn = AccuracyEstimator(topk=(1, ))
        #self.optimizer = optim.SGD(
        #    self.model.parameters(),
        #    lr=optimizer_config.learning_rate,
        #    weight_decay=optimizer_config.weight_decay,
        #    momentum=optimizer_config.momentum
        #)
        #self.lr_scheduler = MultiStepLR(
        #    self.optimizer, milestones=optimizer_config.lr_step_milestones, gamma=optimizer_config.lr_gamma
        #)
        self.optimizer = optim.Adam(self.model.parameters())
        self.lr_scheduler = lr_scheduler.ReduceLROnPlateau(self.optimizer)
        self.visualizer = TensorBoardVisualizer()

    def run(self, trainer_config: configuration.TrainerConfig) -> dict:

        device = torch.device(trainer_config.device)
        self.model = self.model.to(device)
        self.loss_fn = self.loss_fn.to(device)

        model_trainer = Trainer(
            model=self.model,
            loader_train=self.loader_train,
            loader_test=self.loader_test,
            loss_fn=self.loss_fn,
            metric_fn=self.metric_fn,
            optimizer=self.optimizer,
            lr_scheduler=self.lr_scheduler,
            device=device,
            data_getter=itemgetter(0),
            target_getter=itemgetter(1),
            stage_progress=trainer_config.progress_bar,
            get_key_metric=itemgetter("top1"),
            visualizer=self.visualizer,
            model_saving_frequency=trainer_config.model_saving_frequency,
            save_dir=trainer_config.model_dir
        )

        model_trainer.register_hook("end_epoch", hooks.end_epoch_hook_classification)
        self.metrics = model_trainer.fit(trainer_config.epoch_num)
        return self.metrics


In [ ]:
from tqdm import tqdm
class Test:
    def __init__(
        self,
        system_config: configuration.SystemConfig = configuration.SystemConfig(),
        dataset_config: configuration.DatasetConfig = configuration.DatasetConfig(),
        dataloader_config: configuration.DataloaderConfig = configuration.DataloaderConfig(),
    ):
        # test dataloader
        test_dataset = KenyanFood13Dataset(dataset_config.root_dir, flag=2,
                                           split=1.0,
                                           transform=dataset_config.test_transforms, 
                                           random_state=system_config.seed)
        self.loader_test = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=dataloader_config.batch_size,
            shuffle=False,
            num_workers=dataloader_config.num_workers
        )
        self.idx_to_labels = test_dataset.idx_to_labels
        self.model = pretrained_resnext50(pretrained=False)
        self.model.load_state_dict(torch.load('test/model_34_0.872')) # best model saved
        
    def run(self, trainer_config: configuration.TrainerConfig):
        device = torch.device(trainer_config.device)
        
        # set model to eval
        self.model.eval()
        self.model = self.model.to(device)
        data_getter=itemgetter(0)
        imgname_getter=itemgetter(1)
        iterator = tqdm(self.loader_test, disable=not trainer_config.progress_bar, 
                        dynamic_ncols=True)
        preds = []
        imgs = []
        for i, sample in enumerate(iterator):
            inputs = data_getter(sample).to(device)
            names = imgname_getter(sample)
            with torch.no_grad():
                predict = self.model(inputs)
            _, predict = torch.max(predict.cpu(), dim=1)
            #predict = [self.idx_to_labels[i] for i in predict.tolist()]
            preds.extend(predict.tolist())
            imgs.extend(names)
        
        preds = [self.idx_to_labels[i] for i in preds]
        preds = np.c_[preds]
        imgs = np.c_[imgs]
        df = pd.DataFrame({'id': imgs[:,0], 'class': preds[:,0]})
        df.to_csv('submission.csv', index=False)                

def main():
    '''Run the experiment
    '''
    # patch configs depending on cuda availability
    dataloader_config, trainer_config = patch_configs(epoch_num_to_set=15)
    dataset_config = configuration.DatasetConfig(root_dir=".")
    #experiment = Experiment(dataset_config=dataset_config, dataloader_config=dataloader_config)
    #results = experiment.run(trainer_config)
    test = Test(dataset_config=dataset_config, dataloader_config=dataloader_config)
    test.run(trainer_config)

## <font style="color:green">8. TensorBoard Dev Scalars Log Link [5 Points]</font>

Share your tensorboard scalars logs link in this section. You can also share (not mandatory) your GitHub link if you have pushed this project in GitHub. 

For example, [Find Project2 logs here](https://tensorboard.dev/experiment/kMJ4YU0wSNG0IkjrluQ5Dg/#scalars).

https://tensorboard.dev/experiment/SLygBjjvQFmUWk11b2wLPg/#scalars

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

Share your Kaggle profile link here with us so that we can give points for the competition score. 

You should have a minimum accuracy of `75%` on the test data to get all points. If accuracy is less than `70%`, you will not get any points for the section. 

**You must have to submit `submission.csv` (prediction for images in `test.csv`) in `Submit Predictions` tab in Kaggle to get any evaluation in this section.**

https://www.kaggle.com/shivsaxena